<h2>Desafio 2 - Machine Learning


<h3>Grupo 7</h3>
<ul>
    <li>Ignacio Mendieta</li>
    <li>Laura Jazmín Chao</li>
    <li>Juan Nicolás Capistrano</li>
    <li>Betiana Srur</li>
    <li>Marecelo Carrizo</li>
    
</ul>
    

<h4>Objetivo</h4>
Aplicar las ténicas de Machine Learning sobre el dataset de Properati para calcular automáticamente el precio por metro cuadrado de las propiedades. 

<a id="section_toc"></a> 
<h2> Tabla de Contenidos </h2>

[Librerías](#section_import)

[Dataset](#section_dataset)

[Selección de los datos](#section_selection)

[Limpieza de datos](#section_limpieza)

[Imputación de datos faltantes](#section_imput)

$\hspace{.5cm}$[Imputamos datos de <strong>surface</strong>](#section_surface)

$\hspace{.5cm}$[Imputamos datos de <strong>price_usd_per_m2</strong>](#section_price)

$\hspace{.5cm}$[Creación de datos para imputación de rooms](#section_categories)
       
[Extracción de datos de <strong>description</strong>](#section_description)   
    
$\hspace{.5cm}$[Cantidad de ambientes](#section_amb)

$\hspace{.5cm}$[Amenities](#section_caba_description_amenities)
    
[Creación de dummies sobre datos catgóricos](#section_dummies)
    
[Mapa de correlación de datos cuantitativos](#section_corrheatmap)   
    
[Modelos de Regresión](#section_model)
    
$\hspace{.5cm}$[Primeras pruebas](#section_test1)  
    
$\hspace{.5cm}$[Pruebas con Lasso](#section_test_lasso1)  

[Definición de Outliers](#section_outliers)  

[Pruebas con datos ajustados](#section_test2)

$\hspace{.5cm}$[Pruebas Modelo Lasso](#section_test_lasso2)

$\hspace{.5cm}$[Pruebas con modelo Ridge](#section_test_ridge)

$\hspace{.5cm}$[Pruebas con modelo Elastic](#section_test_elastic)

[Creación de features para complejizar el modelo](#section_new_features)

$\hspace{1.cm}$[Superficie2](#section_sup2)

$\hspace{1.cm}$[Superficie3](#section_sup3)

[Estimación de precio total utilizando el mismo modelo](#section_total_price_estimate)

<h3> Checklist </h3>

<h4> De los datos filtrados según Desafio 1 deberiamos:</h4>

<font color='red'>OK</font>  1. Terminar de dropear variables incompletas que serian necesarias para poder entrenar un modelo. (ejemplo: si la propiedad no posee: price, price_aprox_local_currency,price_usd_per_m2, price_per_m2, etc...) la información de los barrios ya fue analizado y dropeado en el paso anterior.

<font color='red'>OK</font> 2. Definir las variables CUANTITATIVAS, ejemplo precio, total de m2 de superficie.

<font color='red'>OK</font> 3. Armar las variables CUALITATIVAS, Dummy (ejemplo: place_name, property_type, amenities, rooms).

<font color='red'>OK</font> 4. Armar una nueva variable Dummy que contemple los precios de propiedad segun su distribución por metro cuadrado. Utilizando Panda CUT.

5. Arreglar la columna rooms para que se pueda usar como numérica en los modelos


<h4> Finalizado el analisis de datos, armamos diferentes modelos para predecir el precio de la propiedad</h4>

<font color='red'>OK</font> 1. Regresion Lineal Multiple

<font color='red'>OK</font> 2. Entrenar los modelos con Lasso, RidgeCV y utilizando Cross Validation.

3. Cambiar los hiperparámetros en Ridge y Lasso???

<a id="section_import"></a> 
<h3>Librerías</h3>

[volver a TOC](#section_toc)

In [ ]:
import numpy as np
import pandas as pd
import re

# Librería Grafica.
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

#Opciones de visualización de columnas y filas al ejecutar las celdas
pd.set_option('display.max_columns', 100) # Para mostrar todas las columnas
pd.set_option('display.max_rows', 100) # Para mostrar todas las filas

<a id="section_dataset"></a> 
<h3>Dataset</h3>

[volver a TOC](#section_toc)

In [ ]:
# Definimos la ruta de la información.
data_propiedades = "Data/properati.csv"

# Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

# Chequeamos que los datos se hayan importado correctamente
data.sample(5)

In [ ]:
# Chequeamos cantidad de registros y cantidad de columnas
data.shape

<a id="section_selection"></a> 
<h3>Selección de los datos</h3>

[volver a TOC](#section_toc)

En base a los análisis previos realizados sobre el Dataset, tomaremos los datos de CABA únicamente para realizar un modelo de machine learning. 

In [ ]:
# Creamos una máscara y la aplicamos al dataframe anterior para traer los registros que necesitamos
data_caba_mask = data.state_name == 'Capital Federal' 
data_caba = data.loc[data_caba_mask, :]

# Chequeamos cómo quedaron los datos
data_caba.head(3)

In [ ]:
data_caba.shape

In [ ]:
data_caba.dtypes

<a id="section_dataset_nulos"></a> 
<h4>Cálculo de cantidad de nulos</h4>

[volver a TOC](#section_toc)

In [ ]:
# # Calculamos cantidad de nulos por campo
# data_caba.isnull().sum()

In [ ]:
# # Calculamos porcentaje de nulos por campo
#  print(np.round((100 * cant_nulos_por_campo / data_caba.shape[0]),2))

<h4> Distribución de <strong>property_type</strong>

In [ ]:
# Contamos cantidad de valores de cada elemento en property_type y operation para ver distribución
property_type_count = data_caba.property_type.value_counts()
print(f'property_type: \n{property_type_count}')

In [ ]:
# # Convertimos el count anterior en las categorías del gráfico
# categories = np.array(property_type_count.index)

# cord_x = data_caba.property_type.value_counts() / data_caba.shape[0]
# palette = ['#440154', '#29788E', '#22A784', '#FDE724']

# p = figure(x_range=categories, plot_width=500, plot_height=200)
# p.vbar(x=categories, top=cord_x, width=0.6,
#        color=palette)

# p.yaxis.formatter = NumeralTickFormatter(format='0 %')
# output_notebook(resources=INLINE)
# show(p)

<h4>Distribución de <strong>currency</strong></h4>

In [ ]:
data_caba.currency.value_counts()

<h4>Distribución de <strong>barrio</strong></h4>

In [ ]:
place_name_count = data_caba.place_name.value_counts()
# place_name_count.sort_index()
# place_name_count

<a id="section_limpieza"></a> 
<h3>Limpieza de datos</h3>

[volver a TOC](#section_toc)

<a id="section_dataset_drop"></a> 
<h4> Drop de columnas innecesarias </h4>

[volver a TOC](#section_toc)

Eliminamos las columnas que en esta instancia no aportan datos útiles para el modelo predictivo que queremos crear. 
 

In [ ]:
# Creamos la lista de columnas a filtrar
drop_columns = ['Unnamed: 0', 'operation', 'place_with_parent_names', 'country_name', 'state_name', 'geonames_id', 'lat-lon','lat', 'lon', 
                'floor', 'expenses','properati_url','title', 'image_thumbnail']

data_caba_clean = data_caba.drop(drop_columns, axis=1)
# data_caba_clean.head(5)


<h3> Limpiamos <strong>currency</strong> </h3>

[volver a TOC](#section_toc)

In [ ]:
# Creamos una mask para traer los valores en monedas que no son USD ni ARS.
currency_OTHER_CURRENCY_mask = (data_caba_clean.currency == 'PEN') | (data_caba_clean.currency == 'UYU')
# Observamos esos registros
data_caba_clean.loc[currency_OTHER_CURRENCY_mask, :]

In [ ]:
# Hacemos drop de esos registros y chequeamos la diferencia entre cantidad original y actual
cant_registros = data_caba_clean.shape[0]
# Dropeamos las moneda PEN y UYU ## OJO la moneda UYU era de una propiedad en Mendoza.
data_caba_clean.drop(data_caba_clean.loc[currency_OTHER_CURRENCY_mask, :].index, inplace = True) 
# Verificamos.
print(f'Cantidad de registros original - registro actuales: {cant_registros - data_caba_clean.shape[0]}')

<h3> Limpiamos <strong>place_name</strong> </h3>

[volver a TOC](#section_toc)

In [ ]:
# Creamos una mascara y dropeamos los de 'Capital Federal'
capi_mask = data_caba_clean.place_name == 'Capital Federal'
data_caba_clean.drop(data_caba_clean.loc[capi_mask, :].index, inplace=True)

# Chequeamos cómo quedaron los datos
data_caba_clean.shape

<h3> Volvemos a calcular cantidad de nulos luego de eliminar columnas y registros sin barrio específico

In [ ]:
# Calculamos cantidad de nulos por campo
data_caba_clean.isnull().sum()

In [ ]:
# Calculamos porcentaje de nulos por campo
print(np.round((100 * data_caba_clean.isnull().sum() / data_caba_clean.shape[0]),2))

In [ ]:
# data_caba_clean.sample(5)

<h3> Eliminamos registros según un umbral de datos faltantes </h3>

[volver a TOC](#section_toc)

In [ ]:
data_caba_clean.shape

In [ ]:
# Eliminamos los registros donde no haya por lo menos 10 campos completos. 

umbral = 10
data_caba_clean.dropna(axis = 0, thresh=umbral, inplace=True)

display(data_caba_clean.shape)
# Se pierden 3500 datos aprox (ver shape de abajo)

In [ ]:
#Vemos cómo quedaron los datos de precio que nos importan
mask_price_validation = data_caba_clean.price != data_caba_clean.price_aprox_usd
display(data_caba_clean.loc[mask_price_validation, :].shape)


# display(data_caba_clean.loc[mask_price_validation, :].head(30))
# data_caba_clean.sample(5)

In [ ]:
data_caba_clean.shape

In [ ]:
data_caba_clean.isnull().sum()

<a id="section_imput"></a> 
<h3>Imputación de datos faltantes</h3>

[volver a TOC](#section_toc)

Primero realizamos ciertas verificaciones de los datos

In [ ]:
# 1. Verificamos que el price y el price_aprox_usd sean los mismos, y luego si fuera así dropeamos uno de los dos y utilizamos el otro como target

mask_price_validation = data_caba_clean.price != data_caba_clean.price_aprox_usd

display(data_caba_clean.loc[mask_price_validation, :].shape)

# Detectamos que hay 1131 propiedades donde los valores son diferentes? pero estan en Pesos o son NaN?

display(data_caba_clean.loc[mask_price_validation, :].sample(10))

# Detectamos que tenemos los dos casos, donde los precios son NaN y donde la moneda está en ARS (lo cual no estaría mal)
# En el caso de ARS, vemos que el valor de price toma el de la moneda local ~ Por lo que podriamos llegar a eliminar price, currency y price_aprox_local_currency 

# Para los valores de NaN tendriamos que tomar la decisión si los dropeamos o tratamos de calcular un precio promedio en base a la cantidad de metros cuadrados.

<a id="section_surface"></a> 
<h3> Completamos datos faltantes de superficie </h3>

[volver a TOC](#section_toc)

In [ ]:
# Para la superficie de metros cuadadros totales, podemos creer que en caso de ser Nan 
# es porque es un departamente sin balcon y la cantidad de m2 cubierta sería iguala la cantidad de m2 totales.data_caba

data_caba_clean.surface_total_in_m2.fillna(data_caba_clean.surface_covered_in_m2 + (data_caba_clean.surface_covered_in_m2 * 0.10), inplace=True)

# data_caba_clean.sample(5)

In [ ]:
# Idem al punto anterior pero ahora de surface_total a surface_covered

data_caba_clean.surface_covered_in_m2.fillna(data_caba_clean.surface_total_in_m2, inplace=True)

# data_caba_clean.sample(5)

<a id="section_price"></a> 
<h3> Completamos datos faltantes de price_usd_per_m2 </h3>

[volver a TOC](#section_toc)

In [ ]:
# Idem al punto anterior pero ahora calculamos el precio por metro cuadrado.

data_caba_clean.price_usd_per_m2.fillna(data_caba_clean.price_aprox_usd / data_caba_clean.surface_total_in_m2, inplace=True)

data_caba_clean.sample(5)

<h3> Eliminamos columnas de precios en moneda local </h3>

[volver a TOC](#section_toc)

In [ ]:
# Dropeamos los valores que sabemos que está repetidos o con otra moneda.data_caba

data_caba_clean.drop(columns=['price', 'currency','price_per_m2', 'price_aprox_local_currency'], inplace=True)

data_caba_clean.sample(10)

<h3> Nuevo recuento de nulos </h3>

In [ ]:
print('\nPorcentaje de valores incompletos por columna:')
print(f'{round(100 * data_caba_clean.isnull().sum()/data_caba_clean.shape[0], 2)}')

display(data_caba_clean.shape)

<a id="section_categories"></a> 
<h3>Creación de datos para imputación de <strong>rooms<strong/></h3>

[volver a TOC](#section_toc)

In [ ]:
# Delimitamos los bins para realizar un cut
# bins = [20, 30, 45, 75, 150, 220]
bins = [0, 30, 45, 75, 150, 220, 500]
labels = ['mono', 's45', 's75', 's150', 's220', 's500' ]

data_caba_clean['m2_categories'] = pd.cut(data_caba_clean.surface_covered_in_m2, bins, labels)
data_caba_clean['m2_labels'] = pd.cut(x=data_caba_clean.surface_covered_in_m2, bins=bins, labels=labels, right=False)


In [ ]:
data_caba_clean.m2_categories.value_counts().sort_index()

In [ ]:
data_caba_clean.m2_labels.value_counts().sort_index()

<a id="section_description"></a> 
<h3> Extracción de datos del campo <strong>description</strong></h3>

[volver a TOC](#section_toc)

Normalización del campo para que sea todo minúscula.

In [ ]:
#pasamos todos los string de la columna a minúscula para que las expresiones regulares no necesiten contener las dos formas
data_caba_clean.description = data_caba_clean.description.str.lower()

data_caba_clean.description.sample(10)

<a id="section_amb"></a> 
<h4> Cantidad de ambientes</h4>

[volver a TOC](#section_toc)

In [ ]:
#Creamos expresiones regulares y las buscamos en el campo descripción, luego asiganmos los valores encontrados a columnas nuevas

In [ ]:
amb_pattern = "((?P<numero_amb>\d)(\s)?(amb|anv|amv|anb))"
amb_pattern_regex = re.compile(amb_pattern)

In [ ]:
amb_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      amb_pattern_regex.search(x))
mask_amb_match_notnull = amb_match.notnull()

# Aplicamos el grupo que encuentra el dígito de cantidad de ambientes, casteado como integer
data_caba_clean.loc[mask_amb_match_notnull, 'number_rooms'] = \
amb_match.loc[mask_amb_match_notnull].apply(lambda x: int(x.group("numero_amb")))

In [ ]:
dorm_pattern = "((?P<numero_dorm>\d)(\s)?(dorm))"
dorm_pattern_regex = re.compile(dorm_pattern)

In [ ]:
dorm_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else dorm_pattern_regex.search(x))

mask_dorm_match_notnull = dorm_match.notnull()

data_caba_clean.loc[mask_dorm_match_notnull, 'number_rooms'] = \
dorm_match.loc[mask_dorm_match_notnull].apply(lambda x: int(x.group("numero_dorm"))+1)

#se asume que el numero de ambientes es numero_dorm +1

Tratamos de llenar los rooms que faltan con lo que obtuvimos de la descripción

In [ ]:
# Cantidad de nulos que quedan en 'rooms'
data_caba_clean.rooms.isnull().sum()

In [ ]:
# Cantidad de datos que obtuvimos de la descripción
data_caba_clean.number_rooms.notnull().sum()

In [ ]:
data_caba_clean.rooms.fillna(data_caba_clean.number_rooms, inplace=True)

In [ ]:
data_caba_clean.rooms.isnull().sum()

Tratamos de llenar los rooms que faltan con los datos de las categorías creadas

In [ ]:
#Creamos una serie donde el index son las categorias y los valores son la cantidad de ambientes estimados
room_index = ["mono", "s45", "s75", "s150", "s220", 's500']
room_values = [1,2,3,4,5,10]
rooms_series = pd.Series(room_values,index=room_index, dtype=int)
rooms_series.dtypes

In [ ]:
mask_room_null = data_caba_clean.rooms.isnull()


In [ ]:
data_caba_clean.loc[mask_room_null, 'rooms'] = data_caba_clean.m2_labels.loc[mask_room_null].apply(rooms_series)



In [ ]:
data_caba_clean.rooms.value_counts()

In [ ]:
# data_caba_clean.rooms.fillna(data_caba_clean.rooms_upon_categories, inplace=True)
data_caba_clean.rooms.isnull().sum()

In [ ]:
# Vemos qué superficies tienen los registros donde room == null 
mask_room_null = data_caba_clean.rooms.isnull()
data_room_null = data_caba_clean.loc[mask_room_null, 'surface_covered_in_m2']

# plt.hist(data_room_null, bins=40, rwidth=0.8, color = '#29788E')
# plt.xlabel = 'Superficie cubierta'
# plt.ylabel = 'Count'
# plt.show()

data_room_null.value_counts()

In [ ]:
#Eliminamos la columna que ya no vamos a utilizar
data_caba_clean.drop(columns=['number_rooms'], inplace=True)

In [ ]:
data_caba_clean.isnull().sum()

In [ ]:
#Eliminamos los registros que quedan ya que corresponden a data extrema
data_caba_clean.dropna(axis=0, inplace=True)

In [ ]:
#Convertimos los datos de ambientes a numérico nuevamente para poder usarlo como variable
data_caba_clean.rooms = data_caba_clean.rooms.apply(pd.to_numeric)

In [ ]:
data_caba_clean.dtypes

<a id="section_caba_description_amenities"></a> 
<h4> Amenities</h4>

[volver a TOC](#section_toc)

<h5> Pileta </h5>

In [ ]:
pileta_pattern = "(?P<pileta>pileta|piscina|picina|pisina|pool)"
pileta_pattern_regex = re.compile(pileta_pattern)

In [ ]:
pileta_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      pileta_pattern_regex.search(x))
mask_pileta_match_notnull = pileta_match.notnull()

data_caba_clean.loc[mask_pileta_match_notnull, 'pool'] = 1

In [ ]:
# data_caba_clean['pool']

In [ ]:
data_caba_clean['pool'].notnull().sum()

In [ ]:
#Completamos los datos NaN con ceros (convertimos en una variable dummy)
data_caba_clean.pool.fillna(0, inplace = True)

<h5> Laundry </h5>

In [ ]:
laudry_pattern = "(?P<laundry>laundry|lavadero)"
laundry_pattern_regex = re.compile(laudry_pattern)

laundry_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      laundry_pattern_regex.search(x))
mask_laundry_match_notnull = laundry_match.notnull()

data_caba_clean.loc[mask_laundry_match_notnull, 'laundry'] = 1

In [ ]:
data_caba_clean['laundry'].notnull().sum()

In [ ]:
data_caba_clean.laundry.fillna(0, inplace = True)

<h5> Parking </h5>

In [ ]:
parking_pattern = "(?P<parking>parking|estacionamiento|garage|cochera|garaje)"
parking_pattern_regex = re.compile(parking_pattern)

parking_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      parking_pattern_regex.search(x))
mask_parking_match_notnull = parking_match.notnull()

data_caba_clean.loc[mask_parking_match_notnull, 'parking'] = 1

In [ ]:
data_caba_clean['parking'].notnull().sum()

In [ ]:
data_caba_clean.parking.fillna(0, inplace = True)

<h5> Gimnasio </h5>

In [ ]:
gym_pattern = "(?P<gimnasio>gim|gym|fitness|fitnes|ejercicio)"
gym_pattern_regex = re.compile(gym_pattern)

gym_match = data_caba_clean.description.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      gym_pattern_regex.search(x))
mask_gim_match_notnull = gym_match.notnull()

data_caba_clean.loc[mask_gim_match_notnull, 'gym'] = 1

In [ ]:
data_caba_clean['gym'].notnull().sum()

In [ ]:
data_caba_clean.gym.fillna(0, inplace = True)

<a id="section_dummies"></a> 
<h3> Creación dummies sobre datos categóricos </h3>

[volver a TOC](#section_toc)

In [ ]:
# Usamos la función get_dummies con one-hot encoding (drop_first=True) para todas las variables que necesitamos
property_type_dummies = pd.get_dummies(data_caba_clean['property_type'], drop_first = True, prefix='prop_type')
property_type_dummies

In [ ]:
m2_labels_dummies = pd.get_dummies(data_caba_clean['m2_labels'], drop_first = True, prefix='m2_label')

In [ ]:
place_name_dummies = pd.get_dummies(data_caba_clean['place_name'], drop_first = True, prefix='place_name')

In [ ]:
rooms_dummies = pd.get_dummies(data_caba_clean['rooms'], drop_first = True, prefix = 'rooms')

In [ ]:
# Hacemos un join para concatenar las columnas al dataframe
data_caba_clean = data_caba_clean.join([property_type_dummies,m2_labels_dummies, place_name_dummies,rooms_dummies])

In [ ]:
data_caba_clean.head(3)

In [ ]:
# Exportamos el archivo para usar en otras notebooks, esto lo guarda en el mismo directorio en el que estamos operando
# data_caba_clean.to_csv('data_caba_clean.csv')

<a id="section_corrheatmap"></a> 
<h3>Mapa de correlación de varaibles cuantitativas</h3>

[volver a TOC](#section_toc)

In [ ]:
display(data_caba_clean.columns)

In [ ]:
data_caba_clean.shape

In [ ]:
cols = ['price_aprox_usd', 'surface_total_in_m2','surface_covered_in_m2', 
        'price_usd_per_m2', 'rooms','pool', 'laundry', 'parking', 'gym']
#Mapa de correlación

g = plt.figure(figsize=(13,10))
g = sns.heatmap(data_caba_clean[cols].corr(),annot=True, cmap="YlGnBu")
g.set_xticklabels(data_caba_clean[cols].columns, rotation=30)
plt.title("Mapa de calor de correlación entre variables")
plt.show()


<a id="section_model"></a> 
<h3>Modelos de Regresión Lineal</h3>

[volver a TOC](#section_toc)

In [ ]:
#Importamos las librerías de machine learning
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn import metrics, linear_model

In [ ]:
# Funcion para realizar el entrenamiento con un set de testeo.

# Visualización de datos predichos para incluir en las funciones
def graf_lineal_model(X, y, y_pred):
    # Graficamos el modelo
    plt.scatter(y_pred, y, s=30, alpha=0.4, c='b')
    plt.plot(y,y, '-.',c='g')
    plt.show()
    print(y.shape)
    print(y_pred.shape)
    return

#Modelo de regresión lineal
def train_test_error(feature_cols, target):
    X = data_caba_clean[feature_cols]
    y = data_caba_clean[target].values.ravel()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=10)
    
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    
#   print (feature_cols)
#   print ("y_test sample: ",y_test.values[0:10])
#   print ("y_test sample: ",y_test)
#   print ("y_pred sample: ",y_pred[0:20].astype(int))
#   print ('MAE:', metrics.mean_absolute_error(y_test, y_pred))
#   print ('MSE:', metrics.mean_squared_error(y_test, y_pred))
    print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print ('R2:', metrics.r2_score(y_test, y_pred))
    graf_lineal_model(X, y_test, y_pred)
    return 

def train_test_error_lasso(feature_cols, target):
    X = data_caba_clean[feature_cols]
    y = data_caba_clean[target].values.ravel()
    
#   cv = KFold(5, shuffle=True)
    cv = 3
    # OJO que no trabajamos con tiempo por lo tanto debe ser shuffle=True.
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)
    
    model = linear_model.LassoCV(alphas=np.linspace(0.00001,0.1, 1000), normalize=True).fit(X_train, y_train)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
    
    y_pred = model.predict(X_train)
    print(dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum()))
    print ("r^2:", model.score(X, y))
    print ("Imprime Alpha", model.alpha_)
    graf_lineal_model(X, y_train, y_pred)
    return 

def train_test_error_ridge(feature_cols, target):
    X = data_caba_clean[feature_cols]
    y = data_caba_clean[target].values.ravel()
    
#   cv = KFold(5, shuffle=True)
    cv = 3
    # OJO que no trabajamos con tiempo por lo tanto debe ser shuffle=True.
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)
    
    #model = linear_model.Ridge(alpha=0.5, normalize=True).fit(X_train, y_train)
    model = linear_model.RidgeCV(alphas=np.linspace(0.00001,0.1, 1000), normalize=True).fit(X_train, y_train)#agr
    
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
    
    y_pred = model.predict(X_train)
    print(dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum()))
    print ("r^2:", model.score(X, y))
    print ("Coeficientes de la regresión", model.coef_)
    graf_lineal_model(X, y_train, y_pred)
    return 

def train_test_error_elastic(feature_cols, target):
    X = data_caba_clean[feature_cols]
    y = data_caba_clean[target].values.ravel()
#     cv = KFold(5, shuffle=True)
    cv = 3
    # OJO que no trabajamos con tiempo por lo tanto debe ser shuffle=True.
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)
    
    model = linear_model.ElasticNetCV(l1_ratio = 1, max_iter=3000, normalize=True).fit(X_train, y_train)
    
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
    
    y_pred = model.predict(X_train)
    print(dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum()))
    print ("r^2:", model.score(X, y))
    print ("Coeficientes de la regresión", model.coef_)
    graf_lineal_model(X, y_train, y_pred)
    return 

<a id="section_variables"></a> 
<h3>Variables</h3>

[volver a TOC](#section_toc)

Armamos objetos con las listas de variables cualitativas para poder utilizarlas más económicamente

In [ ]:
display(data_caba_clean.columns)

In [ ]:
feature_cols_places = list(place_name_dummies.columns)
feature_cols_prop = list(property_type_dummies.columns)
feature_cols_labels = list(m2_labels_dummies.columns)
feature_cols_rooms = list(rooms_dummies.columns)
# feature_cols = ['price_aprox_usd','surface_covered_in_m2','surface_total_in_m2',
#                'pool', 'laundry', 'parking', 'gym']

target = ['price_usd_per_m2']


<a id="section_test1"></a> 
<h3>Primeras pruebas de regresión simple y múltiple</h3>

[volver a TOC](#section_toc)

In [ ]:
# Regresion lineal Simple
feature_cols = ['price_aprox_usd']
train_test_error(feature_cols, target)

In [ ]:
feature_cols = ['surface_covered_in_m2']
train_test_error(feature_cols, target)

In [ ]:
#Agregamos algunas variables cuantitativas y pobamos distintas combinaciones
feature_cols = ['price_aprox_usd', 'surface_covered_in_m2', 'surface_total_in_m2']
train_test_error(feature_cols, target)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_covered_in_m2', 'surface_total_in_m2', 'rooms']
train_test_error(feature_cols, target)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
                'pool', 'laundry', 'parking', 'gym']
train_test_error(feature_cols, target)

In [ ]:
#Agregamos las dummies de place_name como variables
feature_cols.extend(feature_cols_places)
train_test_error(feature_cols, target)

In [ ]:
feature_cols = ['surface_covered_in_m2']
feature_cols.extend(feature_cols_places)
train_test_error(feature_cols, target)

In [ ]:
#agregamos también el tipo de propiedad
feature_cols.extend(feature_cols_prop)
train_test_error(feature_cols, target)

Observamos que al agregar más variables mejora un poco el resultado del R2 (ya no es negativo) pero de todas maneras es muy bajo.

<a id="section_test_lasso1"></a> 
<h3>Pruebas con modelo Lasso</h3>

[volver a TOC](#section_toc)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
                'pool', 'laundry', 'parking', 'gym']
train_test_error_lasso(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_places) 
train_test_error_lasso(feature_cols, target)

In [ ]:
# feature_cols = ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
#                 'pool', 'laundry', 'parking', 'gym', 'place_name_Puerto Madero']
# train_test_error_lasso(feature_cols, target)

<a id="section_outliers"></a> 
<h3>Definción de outliers</h3>

[volver a TOC](#section_toc)

Luego de estas pruebas, notamos que es necesario definir y eliminar Outiliers tanto en el target como en algunas de las variables

<h4>Price_usd_per_m2</h4>

In [ ]:
fig, ax = plt.subplots()
bp = ax.boxplot(data_caba_clean['price_usd_per_m2'].dropna(), vert=False, showmeans=True)

In [ ]:
#Grafico de distribución de la variable target antes de eliminar datos extremos
plt.hist(data_caba_clean.price_usd_per_m2, bins=20, rwidth=0.8, color = '#29788E')
plt.title('Distribución de Precio por M2')
plt.xlabel('Precio por M2')

plt.show()

In [ ]:
# Definimos outliers aproximados
lower_bound = 0.01
upper_bound = 0.96
rest = data_caba_clean.price_usd_per_m2.quantile([lower_bound, upper_bound])
rest

In [ ]:
# Creamos una mask entendiendo que los precios minimos y maximos pueden rondar los 600 USD y 4000 USD y fijamos outliers aproximados

precioxm2_mask = (data_caba_clean.price_usd_per_m2 >= rest[0.01]) & (data_caba_clean.price_usd_per_m2 <= rest[0.96])
data_caba_clean = data_caba_clean.loc[precioxm2_mask, :]

# Volvemos a graficar

plt.hist(data_caba_clean.price_usd_per_m2, bins=20, rwidth=0.8, color = '#29788E')
plt.title('Distribución de Precio por M2')
plt.xlabel('Precio por M2')

plt.show()

print(data_caba_clean.price_usd_per_m2.max()) 
print(data_caba_clean.price_usd_per_m2.min())

# Parece una distribución normal? sino fuera por los valores elevados que tenemos a partir de los 4000...

<h4>Surface_covered_in_m2</h4>

In [ ]:
fig, ax = plt.subplots()
bp = ax.boxplot(data_caba_clean['surface_covered_in_m2'].dropna(), vert=False, showmeans=True)

In [ ]:
#Grafio de la distribucion de superficie antes de eliminar outliers
plt.hist(data_caba_clean.surface_covered_in_m2, bins=20, rwidth=0.8, color = '#29788E')
plt.title('Distribución de Superficie cubierta')
plt.xlabel('Superficie cubierta en M2')
plt.show()

In [ ]:
# Definimos outliers aproximados
lower_bound = 0.001
upper_bound = 0.99
rest = data_caba_clean.surface_covered_in_m2.quantile([lower_bound, upper_bound])
rest

In [ ]:
surface_xm2_mask = (data_caba_clean.surface_covered_in_m2 >= rest[0.001]) & (data_caba_clean.surface_covered_in_m2 <= rest[0.99])
data_caba_clean = data_caba_clean.loc[surface_xm2_mask, :]

# Graficamos la distribución actual de superficie cubierta

plt.hist(data_caba_clean.surface_covered_in_m2, bins=20, rwidth=0.8, color = '#29788E')
plt.title('Distribución de Superficie cubierta')
plt.xlabel('Superficie cubierta en M2')
plt.show()

print(data_caba_clean.surface_covered_in_m2.max()) 
print(data_caba_clean.surface_covered_in_m2.min())

# Parece una distribución normal? sino fuera por los valores elevados que tenemos a partir de los 3500...

In [ ]:
#Nuevo mapa de correlación
X = data_caba_clean[['price_usd_per_m2','price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
        'pool', 'laundry', 'parking', 'gym', 'rooms', 'place_name_Puerto Madero']]

g = plt.figure(figsize=(13,10))
g = sns.heatmap(X.corr(),annot=True, cmap="YlGnBu")
g.set_xticklabels(X.columns, rotation=45)
plt.title("Correlation Heatmap")
plt.show()

Volvemos a definir las dummies de 'rooms' ya que cambiaron los valores de superficie

In [ ]:
rooms_dummies = pd.get_dummies(data_caba_clean['rooms'], drop_first = True, prefix = 'rooms')

In [ ]:
feature_cols_rooms =list(rooms_dummies.columns)

<a id="section_test2"></a> 
<h3>Nuevas pruebas con datos ajustados</h3>

[volver a TOC](#section_toc)

In [ ]:
#Regresión lineal simple con datos ajustados. Aún con una sola variable notamos una mejora sustancial
feature_cols = ['price_aprox_usd']
train_test_error(feature_cols, target)

In [ ]:
#Regresión lineal con variables base
feature_cols = ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
                'pool', 'laundry', 'parking', 'gym']
train_test_error(feature_cols, target)

In [ ]:
#Regresión lineal con variables base
feature_cols = ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
                'pool', 'laundry', 'parking', 'gym', 'rooms']
train_test_error(feature_cols, target)

In [ ]:
#Agregamos a las features de arriba las dummies de place_name
feature_cols.extend(feature_cols_places)
train_test_error(feature_cols, target)

In [ ]:
#Agregamos las dummies de prop_type
feature_cols.extend(feature_cols_prop)
train_test_error(feature_cols, target)

In [ ]:
#Prueba de Regresión lineal con todas las dummies y todas las variables
feature_cols.extend(feature_cols_labels)
train_test_error(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_rooms)
train_test_error(feature_cols, target)

<a id="section_test_lasso2"></a> 
<h3>Nuevas pruebas de Modelo Lasso con datos ajustados</h3>

[volver a TOC](#section_toc)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
                'pool', 'laundry', 'parking', 'gym']
train_test_error_lasso(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_places)
train_test_error_lasso(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_prop)
train_test_error_lasso(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_labels)
train_test_error_lasso(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_rooms)
train_test_error_lasso(feature_cols, target)

Los resultados con Lasso son muy parecidos a los de las LinearRegression, pero no la mejoran

<a id="section_test_ridge"></a> 
<h3>Pruebas con modelo Ridge</h3>

[volver a TOC](#section_toc)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_covered_in_m2']
train_test_error_ridge(feature_cols, target)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
                'pool', 'laundry', 'parking', 'gym', 'rooms']
train_test_error_ridge(feature_cols, target)

In [ ]:
#Prueba de Ridge con variables base + dummies de place_name
feature_cols.extend(feature_cols_places)
train_test_error_ridge(feature_cols, target)

In [ ]:
#Prueba de Ridge con variables base + dummies de place_name + dummies de prop_type
feature_cols.extend(feature_cols_prop)
train_test_error_ridge(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_rooms)
train_test_error_ridge(feature_cols, target)

Lo mismo se observa con el modelo Ridge, los resultados obtenidos son muy similares, pero no mejoran el modelo inicial. 

<a id="section_test_elastic"></a> 
<h3>Pruebas con modelo Elastic</h3>

[volver a TOC](#section_toc)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
                'pool', 'laundry', 'parking', 'gym']
train_test_error_elastic(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_places)
train_test_error_elastic(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_prop)
train_test_error_elastic(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_labels)
train_test_error_elastic(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_rooms)
train_test_error_elastic(feature_cols, target)

<a id="section_new_features"></a> 
<h3>Creación de features para complejizar el modelo</h3>

[volver a TOC](#section_toc)

Creamos nuevas features para complejizar el modelo y testeamos nuevamente con todos los modelos

<a id="section_sup2"></a> 
<h4>Superficie<sup>2</sup></h4>

[volver a TOC](#section_toc)

In [ ]:
#Creo una nueva feature a partir de una existente para complejizar el modelo y tratar de obtener un R^2 más elevado
data_caba_clean['surface_covered_in_m2_2']=data_caba_clean['surface_covered_in_m2']**2
data_caba_clean['surface_covered_in_m2_3']=data_caba_clean['surface_covered_in_m2']**3
data_caba_clean.head()

In [ ]:
# tomo las nueva feature 'surface_covered_in_m2_2'
X = data_caba_clean[['price_usd_per_m2','price_aprox_usd', 'surface_total_in_m2','pool', 'laundry', 'parking', 'gym',
                     'surface_covered_in_m2_2','surface_covered_in_m2']]

plt.figure(figsize=(13,10))
sns.heatmap(X.corr(),annot=True)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2','pool', 'laundry', 'parking', 'gym',
                'surface_covered_in_m2','surface_covered_in_m2_2']
train_test_error(feature_cols, target)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2','pool', 'laundry', 'parking', 'gym',
                'surface_covered_in_m2','surface_covered_in_m2_2']
train_test_error_lasso(feature_cols, target)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2','pool', 'laundry', 'parking', 'gym',
                'surface_covered_in_m2','surface_covered_in_m2_2']
train_test_error_ridge(feature_cols, target)

In [ ]:
#Pruebo con todas las features y con la creada surface_covered_in_m2_2
feature_cols.extend(feature_cols_places)
train_test_error(feature_cols, target)

In [ ]:
feature_cols.extend(feature_cols_prop)
train_test_error(feature_cols, target)

In [ ]:
train_test_error_lasso(feature_cols, target)

In [ ]:
train_test_error_ridge(feature_cols, target)

In [ ]:
train_test_error_elastic(feature_cols, target)

<a id="section_sup3"></a> 
<h4>Superficie<sup>3</sup></h4>

[volver a TOC](#section_toc)

In [ ]:
# agrego una nueva feature ('surface_covered_in_m2_3')

In [ ]:
X = data_caba_clean[['price_usd_per_m2','price_aprox_usd', 'surface_total_in_m2','pool', 'laundry', 'parking', 'gym',
                     'surface_covered_in_m2_2','surface_covered_in_m2','surface_covered_in_m2_3']]

plt.figure(figsize=(13,10))
sns.heatmap(X.corr(),annot=True)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2','pool', 'laundry', 'parking', 'gym',
                'surface_covered_in_m2','surface_covered_in_m2_2','surface_covered_in_m2_3']
train_test_error(feature_cols, target)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2','pool', 'laundry', 'parking', 'gym',
                'surface_covered_in_m2','surface_covered_in_m2_2','surface_covered_in_m2_3']
train_test_error_lasso(feature_cols, target)

In [ ]:
feature_cols = ['price_aprox_usd', 'surface_total_in_m2','pool', 'laundry', 'parking', 'gym',
                'surface_covered_in_m2','surface_covered_in_m2_2','surface_covered_in_m2_3']
train_test_error_ridge(feature_cols, target)

In [ ]:
# con todas las features
feature_cols.extend(feature_cols_places)
feature_cols.extend(feature_cols_prop)
feature_cols.extend(feature_cols_rooms)
train_test_error(feature_cols, target)

In [ ]:
train_test_error_lasso(feature_cols, target)

In [ ]:
train_test_error_ridge(feature_cols, target)

In [ ]:
train_test_error_elastic(feature_cols, target)

<a id="section_total_price_estimate"></a> 
<h3>Estimación de precio total utilizando el mismo modelo</h3>

[volver a TOC](#section_toc)

In [ ]:
feature_cols = ['surface_covered_in_m2','surface_total_in_m2',
                'pool', 'laundry', 'parking', 'gym', 'rooms']
feature_cols.extend(feature_cols_places)
feature_cols.extend(feature_cols_prop)
feature_cols.extend(feature_cols_labels)
feature_cols.extend(feature_cols_rooms)

target = ['price_aprox_usd']


In [ ]:
train_test_error(feature_cols, target)

In [ ]:
train_test_error_elastic(feature_cols, target)

In [ ]:
train_test_error_lasso(feature_cols, target)

In [ ]:
train_test_error_ridge(feature_cols, target)